In [1]:
#!pip install pretty_midi

In [2]:
import pretty_midi as pm
import numpy as np

In [5]:
test_midi = pm.PrettyMIDI('Dataset/1.mid')

In [6]:
test_midi

In [7]:
test_midi.instruments

[Instrument(program=0, is_drum=False, name="Nexus 1")]

In [8]:
test_midi.instruments[0].notes

[Note(start=0.000000, end=0.230769, pitch=76, velocity=100),
 Note(start=0.346153, end=0.576922, pitch=76, velocity=100),
 Note(start=0.692307, end=0.923076, pitch=76, velocity=100),
 Note(start=1.038460, end=1.269229, pitch=76, velocity=100),
 Note(start=1.384614, end=1.615383, pitch=74, velocity=100),
 Note(start=1.615383, end=1.846152, pitch=72, velocity=100),
 Note(start=1.846152, end=2.076921, pitch=79, velocity=100),
 Note(start=2.192305, end=2.423074, pitch=79, velocity=100),
 Note(start=2.538459, end=2.769228, pitch=76, velocity=100),
 Note(start=2.884612, end=3.115381, pitch=76, velocity=100),
 Note(start=3.230766, end=3.461535, pitch=74, velocity=100),
 Note(start=3.461535, end=3.692304, pitch=72, velocity=100),
 Note(start=3.692304, end=3.923073, pitch=79, velocity=100),
 Note(start=4.038457, end=4.269226, pitch=79, velocity=100),
 Note(start=4.384611, end=4.615380, pitch=76, velocity=100),
 Note(start=4.730764, end=4.961533, pitch=76, velocity=100),
 Note(start=5.076918, en

In [9]:
test_midi.instruments[0].notes[0].pitch

76

In [10]:
def midi_to_list(midi):
    pitch_list = []
    for note in midi.instruments[0].notes:
        pitch_list.append(note.pitch)
    return pitch_list

def list_shift(pitch_list, lowest, highest):
    pitch_list_new = []
    for note in pitch_list:
        if(note > highest):
            print('note with pitch ' + str(note) + 'spotted')
        pitch_list_new.append(note - lowest)
    return pitch_list_new

In [11]:
list_shift(midi_to_list(test_midi),69,81)#0 - A5, 12 - A6

[7, 7, 7, 7, 5, 3, 10, 10, 7, 7, 5, 3, 10, 10, 7, 7, 5, 3, 7, 7, 5, 5, 3, 2]

In [ ]:
#notes_4_second_opt = {0: 0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:8, 9:9, 10:10, 11:11,12:12, 13:13}

#def first_opt(pitch_list_new):
#    tmp = []
#    for note in pitch_list_new:
#        tmp.append(notes_4_first_opt[note])
#    return tmp

In [32]:
#функция для смещения мелодий на полутон вверх

def semitone_shift(note_list):
    tmp = []
    for note in note_list:
        tmp.append(note + 1)
    return tmp

# Загрузка тренировочного датасета

In [72]:
data_full = np.zeros((34,24))

for num in range(0,34):
    tmp_midi = pm.PrettyMIDI('Dataset/'+ str(num+1) + '.mid')
    tmp_midi_edited = list_shift(midi_to_list(tmp_midi),69,81)
    if(np.mod(num,2) == 1):
        tmp_midi_edited = semitone_shift(tmp_midi_edited)
    data_full[num] = tmp_midi_edited

In [73]:
#теперь нужно преобразовать датасет для нейросети

data_4_nn = np.zeros((34, 24, 14))

for melody in range(0, 34):
    for melody_note in range(0, 24):
            data_4_nn[melody][melody_note][int(data_full[melody][melody_note])] = 1

In [74]:
#Создание последовательностей

X = []
y = []

for melody in range(34):
    for melody_note in range(19):
        X.append([data_4_nn[melody][melody_note],data_4_nn[melody][melody_note+1],data_4_nn[melody][melody_note+2],data_4_nn[melody][melody_note+3],data_4_nn[melody][melody_note+4]])
        y.append(data_4_nn[melody][melody_note+5])

In [75]:
X = np.array(X)
y = np.array(y)

In [76]:
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, LSTM
#from keras.utils import np_utils
#from keras.callbacks import ModelCheckpoint

In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

In [78]:
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [79]:
model.compile(loss='categorical_crossentropy', optimizer='adam') 

In [80]:
filepath = "model_2_weights_saved.hdf5" 
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min') 
desired_callbacks = [checkpoint] 

In [81]:
model.fit(X, y, epochs = 150, batch_size = 64, callbacks=desired_callbacks)

Epoch 1/150
11/11 [==============================] - ETA: 0s - loss: 2.5394
Epoch 00001: loss improved from inf to 2.53936, saving model to model_2_weights_saved.hdf5
11/11 [==============================] - 2s 152ms/step - loss: 2.5394
Epoch 2/150
11/11 [==============================] - ETA: 0s - loss: 2.1591
Epoch 00002: loss improved from 2.53936 to 2.15905, saving model to model_2_weights_saved.hdf5
11/11 [==============================] - 2s 162ms/step - loss: 2.1591
Epoch 3/150
11/11 [==============================] - ETA: 0s - loss: 1.9717
Epoch 00003: loss improved from 2.15905 to 1.97172, saving model to model_2_weights_saved.hdf5
11/11 [==============================] - 2s 147ms/step - loss: 1.9717
Epoch 4/150
11/11 [==============================] - ETA: 0s - loss: 1.9073
Epoch 00004: loss improved from 1.97172 to 1.90731, saving model to model_2_weights_saved.hdf5
11/11 [==============================] - 2s 173ms/step - loss: 1.9073
Epoch 5/150
11/11 [=====================

Epoch 38/150
11/11 [==============================] - ETA: 0s - loss: 1.4025
Epoch 00038: loss improved from 1.41726 to 1.40246, saving model to model_2_weights_saved.hdf5
11/11 [==============================] - 1s 122ms/step - loss: 1.4025
Epoch 39/150
11/11 [==============================] - ETA: 0s - loss: 1.3740
Epoch 00039: loss improved from 1.40246 to 1.37400, saving model to model_2_weights_saved.hdf5
11/11 [==============================] - 1s 121ms/step - loss: 1.3740
Epoch 40/150
11/11 [==============================] - ETA: 0s - loss: 1.3473
Epoch 00040: loss improved from 1.37400 to 1.34728, saving model to model_2_weights_saved.hdf5
11/11 [==============================] - 1s 121ms/step - loss: 1.3473
Epoch 41/150
11/11 [==============================] - ETA: 0s - loss: 1.3592
Epoch 00041: loss did not improve from 1.34728
11/11 [==============================] - 1s 114ms/step - loss: 1.3592
Epoch 42/150
11/11 [==============================] - ETA: 0s - loss: 1.3233
Epo

Epoch 75/150
11/11 [==============================] - ETA: 0s - loss: 0.8524
Epoch 00075: loss did not improve from 0.83594
11/11 [==============================] - 1s 115ms/step - loss: 0.8524
Epoch 76/150
11/11 [==============================] - ETA: 0s - loss: 0.8825
Epoch 00076: loss did not improve from 0.83594
11/11 [==============================] - 1s 114ms/step - loss: 0.8825
Epoch 77/150
11/11 [==============================] - ETA: 0s - loss: 0.8593
Epoch 00077: loss did not improve from 0.83594
11/11 [==============================] - 1s 114ms/step - loss: 0.8593
Epoch 78/150
11/11 [==============================] - ETA: 0s - loss: 0.8672
Epoch 00078: loss did not improve from 0.83594
11/11 [==============================] - 1s 113ms/step - loss: 0.8672
Epoch 79/150
11/11 [==============================] - ETA: 0s - loss: 0.8289
Epoch 00079: loss improved from 0.83594 to 0.82889, saving model to model_2_weights_saved.hdf5
11/11 [==============================] - 1s 132ms/st

Epoch 113/150
11/11 [==============================] - ETA: 0s - loss: 0.4889
Epoch 00113: loss improved from 0.49296 to 0.48891, saving model to model_2_weights_saved.hdf5
11/11 [==============================] - 2s 145ms/step - loss: 0.4889
Epoch 114/150
11/11 [==============================] - ETA: 0s - loss: 0.4517
Epoch 00114: loss improved from 0.48891 to 0.45170, saving model to model_2_weights_saved.hdf5
11/11 [==============================] - 1s 133ms/step - loss: 0.4517
Epoch 115/150
11/11 [==============================] - ETA: 0s - loss: 0.4715
Epoch 00115: loss did not improve from 0.45170
11/11 [==============================] - 1s 127ms/step - loss: 0.4715
Epoch 116/150
11/11 [==============================] - ETA: 0s - loss: 0.4413
Epoch 00116: loss improved from 0.45170 to 0.44128, saving model to model_2_weights_saved.hdf5
11/11 [==============================] - 2s 197ms/step - loss: 0.4413
Epoch 117/150
11/11 [==============================] - ETA: 0s - loss: 0.463

In [82]:
filename = "model_2_weights_saved.hdf5" 
model.load_weights(filename) 
model.compile(loss='categorical_crossentropy', optimizer='adam') 

In [83]:
melody_pred = np.zeros((5))
X_pred = np.zeros((1,5,14))

melody_pred[0] = int(np.round(13*np.random.rand()))
melody_pred[1] = int(np.round(13*np.random.rand()))
melody_pred[2] = int(np.round(13*np.random.rand()))
melody_pred[3] = int(np.round(13*np.random.rand()))
melody_pred[4] = int(np.round(13*np.random.rand()))

for i in range(melody_pred.shape[0]):
    X_pred[0][i][int(melody_pred[i])] = 1
    
X_pred_012 = X_pred

In [84]:
final_midi = []
zeros = np.zeros((14))
for i in range(0,19):
    prediction = np.argmax(model.predict(X_pred, verbose = 0))
    
    final_midi.append(prediction)
    X_pred[0][0] = X_pred[0][1]
    X_pred[0][1] = X_pred[0][2]
    X_pred[0][2] = X_pred[0][3]
    X_pred[0][3] = X_pred[0][4]
    X_pred[0][4] = zeros
    X_pred[0][4][prediction] = 1

final_midi
melody_start = []
for note in melody_pred:
    melody_start.append(int(note))
#melody_start
final_midi = melody_start + final_midi

In [85]:
final_midi

[10, 12, 4, 5, 12, 11, 9, 9, 9, 8, 9, 8, 8, 4, 4, 6, 6, 8, 8, 8, 8, 6, 8, 8]

In [86]:
def shift_back(final_midi, lowest):
    tmp = []
    for note in final_midi:
        tmp.append(note + lowest)
    return tmp

final_shifted = shift_back(final_midi, 69)

In [87]:
export_midi = pm.PrettyMIDI('Dataset/1.mid')

for i in range(24):
    export_midi.instruments[0].notes[i].pitch = final_shifted[i]

export_midi.write('Output_2.mid')

In [88]:
X.shape

(646, 5, 14)